# Cable Tuning Bridge 2D
***
<i>Demonstrate cable tuning analysis of a 2D structure</i>

In [ ]:
''' Inputs '''
tower_base = 6                   # Metres below the origin
tower_height_to_first_stay = 10  # Metres above the origin
cable_spacing_tower = 1          # Metre increments

cable_spacing_forwards = [5,5,5,5,5,5,5,5]
cable_spacing_backwards = [5,5,5,5,5,5,5,5]

deck_width = 5.0
deck_depth = 0.6

tower_width_base = 2.0
tower_depth_base = 2.6
tower_width_top = 2.0
tower_depth_top = 2.6

stay_dia = 0.15

live_load = 20

In [ ]:
''' Data Checks'''
assert len(cable_spacing_backwards) == len(cable_spacing_forwards)

In [ ]:
import numpy as np
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "2D_Cable_Tuning_Analysis")
# Reference to the model database for convenience
db = lusas.database() 
# 2D model with Y vertical
db.setAnalysisCategory("2D Inplane")
db.setVerticalDir("Y")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [ ]:
''' Create Model Points '''
point_origin = Helpers.create_point(0,0,0)
tower_points = [Helpers.create_point(0,-tower_base,0)]
y = tower_height_to_first_stay
for i in range(0, len(cable_spacing_forwards)):
    tower_points.append(Helpers.create_point(0,y,0))
    y+=cable_spacing_tower


deck_points_forwards = [Helpers.create_point(x,0,0) for x in np.add.accumulate(cable_spacing_forwards)]
deck_points_backwards = [Helpers.create_point(-x,0,0) for x in np.add.accumulate(cable_spacing_backwards)]
# All deck points in x order
deck_points = deck_points_backwards[::-1] + deck_points_forwards


In [ ]:
''' Create model lines '''

tower_lines = []
for i in range(0, len(tower_points)-1):
    tower_lines.append(Helpers.create_line_from_points(tower_points[i], tower_points[i+1]))

deck_lines = []
for i in range(0, len(deck_points)-1):
    deck_lines.append(Helpers.create_line_from_points(deck_points[i], deck_points[i+1])) 

back_stays = []
for i in range(0, len(deck_points_backwards)-1):
    back_stays.append(Helpers.create_line_from_points(deck_points_backwards[i], tower_points[i+2])) 

front_stays = []
for i in range(0, len(deck_points_forwards)-1):
    front_stays.append(Helpers.create_line_from_points(deck_points_forwards[i], tower_points[i+2])) 

In [ ]:
''' Create Geometric Sections '''
deck_section_attr = Helpers.create_rectangular_section(db, "Deck", deck_width, deck_depth)
tower_section_attr = Helpers.create_rectangular_section(db, "Tower", tower_width_base, tower_depth_base)
stay_section_attr = Helpers.create_circular_section(db, "Stay", stay_dia)

deck_section_attr.assignTo(deck_lines)
tower_section_attr.assignTo(tower_lines)
stay_section_attr.assignTo(back_stays)
stay_section_attr.assignTo(front_stays)

In [ ]:
''' Create Material Attributes'''

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000_000, 0.3, 7.8, 10e-6)

# Concrete Material attribute
concrete_material = db.createIsotropicMaterial("Concrete", 30_000_000, 0.2, 2.4, 10e-6)

steel_material.assignTo(back_stays)
steel_material.assignTo(front_stays)
concrete_material.assignTo(deck_lines)
concrete_material.assignTo(tower_lines)

In [ ]:
''' Support attributes ''' 
fixed = db.createSupportStructural("Fixed in XY").setStructural("R", "R", "F", "R", "R", "R", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Y").setStructural("F", "R", "F", "F", "F", "F", "F", "F", "C", "F")

fixed.assignTo(tower_points[0])
slide.assignTo(deck_points_backwards[-1])
slide.assignTo(deck_points_forwards[-1])

In [ ]:
''' Mesh Attributes'''
# Cables modelled with single 2D BAR element
cable_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BAR2", 1)
# Deck and tower modelled with 2D thick beam elements (BMI2)
beam_mesh_attr = db.createMeshLine("Beam mesh 1m").setSize("BMI2", 1)

cable_mesh_attr.assignTo(back_stays)
cable_mesh_attr.assignTo(front_stays)
beam_mesh_attr.assignTo(deck_lines)
beam_mesh_attr.assignTo(tower_lines)

# Create the model mesh
db.updateMesh()

# Loads

In [ ]:
# Automatic gravity in loadcase 1
self_weight_loadcase = db.getLoadset("Loadcase 1").setName("Self Weight")
win32.CastTo(self_weight_loadcase, "IFLoadcase").addGravity(True)

In [ ]:
# Create a loadcase for the live load
live_load_loadcase = db.createLoadcase("Live Load")
# Set the loadcase in the default assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(live_load_loadcase)
# Create a load attribute for a distributed beam load
beam_live_load_attr = db.createLoadingBeamDistributed(f"Live Load")
# beam_live_load_attr.setAnalysisCategory("2D Inplane")
beam_live_load_attr.setBeamDistributed("Parametric", "Global", "beam")
beam_live_load_attr.addRow(0.0, 0.0, -live_load, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -live_load, 0.0, 0.0, 0.0, 0.0)
# and assign the load
beam_live_load_attr.assignTo(deck_lines, assignment)

# Set the loadset active
lusas.getCurrentView().setActiveLoadset(live_load_loadcase)


# Cable Tuning Analysis

In [ ]:
# Create a cable tuning analysis
cable_tuning_analysis = db.createAnalysisCableTuning("Cable tuning")
# With an exact solution
cable_tuning_analysis.setOptimisation("Exact")

# Consider all cables
for line in back_stays:
    cable_tuning_analysis.addCable(line.getName(), 1.0)
for line in front_stays:
    cable_tuning_analysis.addCable(line.getName(), 1.0)

# For teh self weight loadcase
cable_tuning_analysis.addEntry("constant", "positive only", 1.0, 1.0, self_weight_loadcase)

# In which we want to find a zero displacement for all points in the deck at the cables.
# Note for an exact solution the number of constraints must macth the number of cables
for i in range(0, len(deck_lines)-1):
    cable_tuning_analysis.addTarget("Point", deck_lines[i].getEndPoint().getID(), "Displacement", "DY", "=", 0.0)


In [ ]:
# Solve the Analyses
db.getAnalysis("Analysis 1").solve(True)
cable_tuning_analysis.solve(True)
db.openAllResults(True)